In [14]:
import cryptography
import secrets
import hashlib
import json

In [5]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

In [6]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

In [7]:
from Crypto.Cipher import AES

### AES SYMMETRIC ENCRYPTION

In [115]:
def random_phrase(length):
    return secrets.token_urlsafe(length)

In [15]:
def aes_sym(message):
    
    # CONVERT DICT TO STRING
    if type(message) == dict:
        message = json.dumps(message)
    
    # GENERATE RANDOM PHRASES
    password = random_phrase(12).encode()
    init_vector = random_phrase(12).encode()
    
    # CREATE CIPHER
    cipher = AES.new(password, AES.MODE_CBC, init_vector)
    
    # PAD MESSAGE TO APPROPARIATE LENGTH
    while len(message) % 16 != 0:
        message += " "
    
    # ENCRYPT ENCODED MESSAGE & RETURN
    encrypted = cipher.encrypt(message.encode())
    
    return encrypted, password

### RSA ASYMMETRIC ENCRYPTION

In [201]:
# https://nitratine.net/blog/post/asymmetric-encryption-and-decryption-in-python/

In [10]:
def generate_keys():
    
    # PRIVATE KEY
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    
    # PUBLIC KEY
    public_key = private_key.public_key()
    
    # RETURN
    return public_key, private_key

In [16]:
def rsa_encrypt(message):
    
    # GENERATE NEW KEYPAIR
    pub_key, priv_key = generate_keys()

    # ENCRYPT WITH PUBKEY
    encrypted = pub_key.encrypt(
        message,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    # PUBLIC KEY PEM
    pub_pem = pub_key.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    # PRIVATE KEY PEM
    priv_pem = priv_key.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    
    return encrypted, pub_pem, priv_pem

In [12]:
def rsa_decrypt(data, priv_key):

    # DECRYPT WITH PRIVKEY
    decrypted = priv_key.decrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return decrypted

### CONVERT KEY FORMATS

In [ ]:
def convert_pubkey(data):
    
    # IF IN BYTE FORMAT
    if type(data) == bytes:
        return serialization.load_pem_public_key(
            data,
            backend=default_backend()
        )
        
    # IF IN BINARY FORMAT
    else:
        return pub_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )

In [11]:
def save_pubkey(pubkey, filename):
    
    # CREATE PEM
    pem = pubkey.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )
    
    # DECODE TO STRINGIFY

    # CREATE FILE
    with open('data/keys/' + filename, 'wb') as file:
        file.write(pem)

In [12]:
def save_privkey(privkey, filename):
    
    # CREATE PEM
    pem = privkey.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )
    
    # DECODE TO STRINGIFY

    # CREATE FILE
    with open('data/keys/' + filename, 'wb') as file:
        file.write(pem)